In [ ]:
import os
!git clone https://github.com/Maria-Elisa-M/BW_prediction_keypoint.git

Mounted at /content/drive


In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.8/921.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 723.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [17]:
import pandas as pd
import numpy as np
import cv2
import os
from itertools import combinations
from ultralytics import YOLO


In [ ]:
os.chdir('BW_prediction_keypoint')
# Define directories
main_dir = 'epth'
image_dir = os.path.join(main_dir,"images")
save_dir =  os.path.join(main_dir,"results")
model_path = os.path.join(main_dir,"keypoint/stream3/train3/weights/best.pt")

In [24]:
# Create save directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Load YOLOv8 pose estimation model
model = YOLO(model_path)

# Define keypoint indices to extract (0-6)
keypoint_indices = list(range(7))

# Prepare DataFrame structure
columns = ['image_name', 'cow', 'day'] + [f"{i}_x" for i in keypoint_indices] + [f"{i}_y" for i in keypoint_indices]
columns += [f"dist_{i}_{j}" for i, j in combinations(keypoint_indices, 2)]
df = pd.DataFrame(columns=columns)

# Process the images in the folder
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)

        # Extract cow and day from filename
        parts = filename.split("_")
        if len(parts) >= 2:
            cow, day = parts[0], parts[1][:8]  # Extract cow and day
        else:
            cow, day = np.nan, np.nan  # Assign NaN if format is incorrect

        # Run inference on the image
        results = model(image_path)[0]  # Get first result

        # Extract keypoints
        if results.keypoints is not None:
            for pose in results.keypoints.xy.cpu().numpy():  # Convert tensor to numpy
                row = [filename, cow, day]  # Start with image filename, cow, and day
                keypoints = []

                for i in keypoint_indices:  # Extract keypoints 0-6
                    if i < len(pose):
                        x, y = int(pose[i][0]), int(pose[i][1])  # Get absolute pixel values
                    else:
                        x, y = np.nan, np.nan  # Assign NaN if keypoint is missing
                    keypoints.append((x, y))
                    row.extend([x, y])  # Append x and y

                # Compute Euclidean distances for all keypoint pairs
                for i, j in combinations(keypoint_indices, 2):
                    if np.nan not in (keypoints[i] + keypoints[j]):  # Ensure both keypoints exist
                        dist = np.sqrt((keypoints[i][0] - keypoints[j][0])**2 +
                                       (keypoints[i][1] - keypoints[j][1])**2)
                    else:
                        dist = np.nan  # Assign NaN if either keypoint is missing
                    row.append(dist)

                # Add row to DataFrame
                df.loc[len(df)] = row

# Save results
output_csv = os.path.join(save_dir, "keypoints_results.csv")
df.to_csv(output_csv, index=False)

print(f"Processing complete. Results saved to {output_csv}.")


#Group the df
def compute_averages(df):
    numeric_columns = df.columns[3:]  # Columns after 'image_name', 'cow', and 'day'

    # Convert relevant columns to numeric to ensure proper averaging
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Group by 'cow' and 'day', then compute the mean
    df_new = df.groupby(['cow', 'day'], as_index=False)[numeric_columns].mean()

    return df_new


df_new = compute_averages(df)


def merge_bw_data(df_new, bw_file_path):
    """
    Merges body weight (bw) data into df_new based on cow and day.

    Parameters:
    df_new (pd.DataFrame): DataFrame with averaged keypoint data.
    bw_file_path (str): Path to the CSV file containing bw data.

    Returns:
    pd.DataFrame: Updated df_new with the bw column merged.
    """
    # Read BW data
    df_bw = pd.read_csv(bw_file_path, dtype={'day': str, 'cow': str})

    # Ensure df_new has the correct data types for merging
    df_new[['day', 'cow']] = df_new[['day', 'cow']].astype(str)

    # Merge the BW data based on 'cow' and 'day'
    df_merged = df_new.merge(df_bw, on=['cow', 'day'], how='left')

    return df_merged






image 1/1 /content/drive/MyDrive/BW_prediction_keypoint/depth/images/4498_20220808_080724_depth_frame_0.png: 384x672 1 cow, 110.9ms
Speed: 2.3ms preprocess, 110.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 672)

image 1/1 /content/drive/MyDrive/BW_prediction_keypoint/depth/images/4487_20220726_083005_depth_frame_0.png: 384x672 1 cow, 104.4ms
Speed: 2.2ms preprocess, 104.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 672)

image 1/1 /content/drive/MyDrive/BW_prediction_keypoint/depth/images/4492_20220815_085944_depth_frame_0.png: 384x672 1 cow, 296.1ms
Speed: 18.1ms preprocess, 296.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 672)

image 1/1 /content/drive/MyDrive/BW_prediction_keypoint/depth/images/4492_20220808_080817_depth_frame_0.png: 384x672 1 cow, 140.8ms
Speed: 2.9ms preprocess, 140.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 672)

image 1/1 /content/drive/MyDrive/BW_prediction_keypoint/depth/images/4489_

In [25]:
# Usage
bw_file_path = os.path.join(main_dir, "DF", "BW.csv")
df_new = merge_bw_data(df_new, bw_file_path)

output_csv = os.path.join(save_dir, "keypoints_results_bw.csv")
df_new.to_csv(output_csv, index=False)

print(f"Processing complete. Results saved to {output_csv}.")

Processing complete. Results saved to /content/drive/MyDrive/BW_prediction_keypoint/depth/results/keypoints_results_bw.csv.
